<a href="https://colab.research.google.com/github/asmakhoualdia98/ortools_pb_solvers/blob/main/sudoku_ortools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.27.1 which is incompatible.
google-ai-generativelanguage 0.6.4 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protob

In [2]:
from ortools.sat.python import cp_model

In [3]:
def sudoku_solver(grid):
    n = len(grid)
    m = int(n**0.5)

    # Create the CP-SAT model
    model = cp_model.CpModel()

    # Variables representing cell values
    cell_vars = {}
    for i in range(n):
        for j in range(n):
          # Creates a 0-1 variable with the given name with NewBoolVar(self, name)
            cell_vars[(i, j)] = [model.NewBoolVar(f'cell_{i}_{j}_{k+1}') for k in range(n)]

    # Constraint 1: Each cell must have exactly one value
    for i in range(n):
        for j in range(n):
            model.Add(sum(cell_vars[(i, j)]) == 1)

    # Constraint 2: Each value must appear exactly once in each row
    for i in range(n):
        for k in range(n):
            model.Add(sum(cell_vars[(i, j)][k] for j in range(n)) == 1)

    # Constraint 3: Each value must appear exactly once in each column
    for j in range(n):
        for k in range(n):
            model.Add(sum(cell_vars[(i, j)][k] for i in range(n)) == 1)

    # Constraint 4: Each value must appear exactly once in each subgrid
    for x in range(0, n, m):
        for y in range(0, n, m):
            for k in range(n):
                model.Add(sum(cell_vars[(i, j)][k] for i in range(x, x + m) for j in range(y, y + m)) == 1)

    # Assign known values
    for i in range(n):
        for j in range(n):
            if grid[i][j] != 0:
                k = grid[i][j] - 1
                model.Add(cell_vars[(i, j)][k] == 1)

    # Create the solver and solve the model
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    # Check the result
    #The status of the solve:

      #* *FEASIBLE* if some solutions have been found
      #* *INFEASIBLE* if the solver has proved there are no solution
      #* *OPTIMAL* if all solutions have been found
    if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
        # Extract the values assigned to each cell
        solution = []
        for i in range(n):
            row = []
            for j in range(n):
                for k in range(n):
                    if solver.Value(cell_vars[(i, j)][k]):
                        row.append(k + 1)
                        break
            solution.append(row)

        # Print the solution
        for row in solution:
            print(row)
    else:
        print("No solution found.")

# Example Sudoku grid (0 represents empty cells)
sudoku_grid = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9]
]

# Solve the Sudoku puzzle
sudoku_solver(sudoku_grid)


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]
